In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Balanced_Another/Client1/train/'

test_data_dir = 'D:/TQ/Balanced_Another/Client1/val/'


In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [5]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 124 samples
Arson: 0 samples
Fight: 342 samples
Normal: 199 samples
Riot: 253 samples
Traffic_Accident: 179 samples
Train_Accident: 150 samples


In [6]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [38]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model1 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [7]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 1001 images belonging to 7 classes.
Found 246 images belonging to 7 classes.
Found 349 images belonging to 7 classes.


In [40]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [41]:
history = model1.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


Epoch 1/5
126/126 [==============================] - 34s 250ms/step - loss: 0.7874 - accuracy: 0.7423 - val_loss: 1.4294 - val_accuracy: 0.5366
Epoch 2/5
126/126 [==============================] - 30s 238ms/step - loss: 0.1922 - accuracy: 0.9371 - val_loss: 1.8712 - val_accuracy: 0.5488
Epoch 3/5
126/126 [==============================] - 28s 219ms/step - loss: 0.1534 - accuracy: 0.9600 - val_loss: 2.1121 - val_accuracy: 0.5610
Epoch 4/5
126/126 [==============================] - 28s 220ms/step - loss: 0.1081 - accuracy: 0.9690 - val_loss: 2.3302 - val_accuracy: 0.5366
Epoch 5/5
126/126 [==============================] - 30s 239ms/step - loss: 0.0746 - accuracy: 0.9820 - val_loss: 2.2035 - val_accuracy: 0.5650


In [42]:
#model = tf.keras.models.load_model("Client1_Centralized.h5")

In [43]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model1.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

349/349 [==============================] - 9s 25ms/step


In [44]:
print("Accuracy:")
test_loss, test_accuracy = model.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:
Test Loss: 1.4517643451690674
Test Accuracy: 0.8280802369117737


In [45]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[30  0  0  1  0  0  0]
 [ 0  0  0 25  0 10  0]
 [ 0  0 73 13  0  0  0]
 [ 0  0  0 43  0  6  1]
 [ 0  0  0  0 64  0  0]
 [ 0  0  0  4  0 41  0]
 [ 0  0  0  0  0  0 38]]


In [46]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix1.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [47]:
model.save("Client1_Centralized.h5")

In [33]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.97      0.98        31
           Arson       1.00      0.46      0.63        35
           Fight       1.00      0.85      0.92        86
          Normal       0.59      0.90      0.71        50
            Riot       1.00      1.00      1.00        64
Traffic_Accident       0.82      1.00      0.90        45
  Train_Accident       1.00      0.92      0.96        38

        accuracy                           0.88       349
       macro avg       0.92      0.87      0.87       349
    weighted avg       0.92      0.88      0.88       349



In [8]:
client1 = tf.keras.models.load_model('D:/TQ/Balanced_Another/client1_30_rounds.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [9]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client1.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client1.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


349/349 [==============================] - 11s 24ms/step
Test Loss: 0.8237746357917786
Test Accuracy: 0.8911174535751343
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.97      0.98        31
           Arson       1.00      0.57      0.73        35
           Fight       1.00      0.85      0.92        86
          Normal       0.58      0.90      0.71        50
            Riot       1.00      1.00      1.00        64
Traffic_Accident       0.90      1.00      0.95        45
  Train_Accident       1.00      0.92      0.96        38

        accuracy                           0.89       349
       macro avg       0.93      0.89      0.89       349
    weighted avg       0.93      0.89      0.90       349



In [10]:
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix After Aggregation:")
print(cm)

Confusion Matrix After Aggregation:
[[30  0  0  1  0  0  0]
 [ 0 20  0 15  0  0  0]
 [ 0  0 73 13  0  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  0  0 64  0  0]
 [ 0  0  0  0  0 45  0]
 [ 0  0  0  3  0  0 35]]


In [37]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix1f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx
